## Sentiment analysis of the lyrics

We will use the same dataframe use to build the network. It has merged songs duplications.

In [1]:
#load de data from .pkl file
import pickle
import numpy as np
import pandas as pd

In [2]:
import os
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import string

In [3]:
from collections import defaultdict

In [4]:
#load de data from .pkl file
loaded_df = pd.read_pickle("ts_data.pkl")


In [5]:
loaded_df.head()

,track_name,track_musical_genre,track_type,duration_ms,feature,track_videoclip,videoclip_views,spotify_streams,spotify_global_peak,album,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,cleaned_lyrics
0,"""Slut!"" (Taylor's Version) (From The Vault)",Synth Pop,Single,180381,No,No,0,239579759,2,"[1989 (Taylor's Version) [Deluxe], 1989 (Taylo...",...,"[0.309, 0.345]","[0.591, 0.629]","[0.412, 0.413]","[0.000199, 7.62e-05]","[0.0608, 0.0603]","[-13.27, -13.178]","[0.0863, 0.055]","[155.875, 77.983]","[0.328, 0.306]",flamingo pink sunrise boulevard clink clink yo...
1,...Ready For It?,Electropop,Single,208186,No,Yes,350.707.005,695895392,3,reputation,...,0.0527,0.613,0.764,0.0,0.197,-6.509,0.136,160.015,0.417,knew killer first time saw wondered many girl ...
2,A Perfectly Good Heart,Country Pop,B-Side,220146,No,No,0,25204096,0,Taylor Swift,...,0.00349,0.483,0.751,0.0,0.128,-5.726,0.0365,156.092,0.268,would wanna break perfectly good heart would w...
3,A Place in this World,Country Pop,B-Side,"[202080, 199200]",No,No,0,40680546,0,[reputation Stadium Tour Surprise Song Playlis...,...,"[0.0577, 0.051]","[0.573, 0.576]","[0.767, 0.777]",0.0,"[0.327, 0.32]","[-2.929, -2.881]","[0.0323, 0.0324]","[114.984, 115.028]","[0.438, 0.428]",dont know want dont ask cause im still trying ...
4,Afterglow,Pop,B-Side,223293,No,No,0,368409481,38,Lover,...,0.13,0.756,0.449,0.0,0.114,-8.746,0.0344,111.011,0.399,blew thing proportion youre blue put jail some...


In [6]:
len(loaded_df)

235

In [21]:
loaded_df.columns

Index(['track_name', 'track_musical_genre', 'track_type', 'duration_ms',
       'feature', 'track_videoclip', 'videoclip_views', 'spotify_streams',
       'spotify_global_peak', 'album', 'track_number', 'album_musical_genre',
       'album_type', 'release_date', 'album_physical_sales', 'track_lyrics',
       'track_theme', 'uri', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'cleaned_lyrics'],
      dtype='object')

In [7]:
#function to calculate sentiment given a list of tokens based on the LabMT wordlist
def sentiment(tokens, LabMT):
    #load the LabMT wordlist from Data_Set_S1.txt
    with open(LabMT, 'r') as f:
        lines = f.readlines()
    #remove the first three lines 
    lines = lines[4:]

    #create a dictionary of words and their happiness values
    word_dict = {}
    for line in lines:
        line = line.split('\t')
        word = line[0]
        happiness = float(line[2])
        word_dict[word] = happiness

    #calculate the sentiment of the tokens
    sentiment = 0
    for token in tokens:
        if token in word_dict:
            sentiment += word_dict[token]
    return sentiment

In [ ]:
#function to calculate the sentiment of the lyrics
def sentiment_lyrics(lyrics, LabMT):
    #lowercase the lyrics
    #lyrics = lyrics.lower()
    #remove punctuation
    #lyrics = lyrics.translate(str.maketrans('', '', string.punctuation))
    #tokenize the lyrics
    #tokens = word_tokenize(lyrics)
    #lemmatize the tokens
    #lemmatizer = WordNetLemmatizer()
    #tokens = [lemmatizer.lemmatize(token) for token in tokens]
    #calculate the sentiment of the tokens
    sentiment_score = sentiment(tokens, LabMT)
    return sentiment_score

In [14]:
#calculate the sentiment of the lyrics.
#Take into account that a song may have a list of lyrics instead of a single string
#create a dictionary with the name of the song as the key and its sentiment as value
LabMT = 'data/Data_Set_S1.txt'
sentiments = defaultdict(list)
for index, row in loaded_df.iterrows():
    lyrics = row['track_lyrics']
    if type(lyrics) == list:
        for lyric in lyrics:
            sentiment_score = sentiment_lyrics(lyric, LabMT)
            sentiments[row['track_name']].append(sentiment_score)
    else:
        sentiment_score = sentiment_lyrics(lyrics, LabMT)
        sentiments[row['track_name']].append(sentiment_score)





In [19]:
#print the length each key of the dictionary
for key in sentiments:
    print(key, len(sentiments[key]))

"Slut!" (Taylor's Version) (From The Vault) 1
...Ready For It? 1
A Perfectly Good Heart 1
A Place in this World 1
Afterglow 1
All Too Well (10 Minute Version) (Taylor's Version) (From The Vault) 1
Anti-Hero 1
Bad Blood 2
Bad Blood (Taylor's Version) 1
Bad Blood (feat. Kendrick Lamar) (Taylor's Version) 1
Beautiful Eyes - Live From Clear Channel Stripped 2008 1
Bejeweled 1
Bigger Than The Whole Sky 1
But Daddy I Love Him 1
Bye Bye Baby (Taylor's Version) (From The Vault) 1
Call It What You Want 1
Cassandra 1
Castles Crumbling (feat. Hayley Williams) (Taylor's Version) (From The Vault) 1
Chloe or Sam or Sophia or Marcus 1
Clara Bow 1
Cold As You 1
Cornelia Street 1
Cruel Summer 1
Dancing With Our Hands Tied 1
Daylight 1
Dear Reader 1
Death By A Thousand Cuts 1
Delicate 1
Don't Blame Me 1
Don't You (Taylor's Version) (From The Vault) 1
Down Bad 1
Dress 1
Electric Touch (feat. Fall Out Boy) (Taylor's Version) (From The Vault) 1
End Game 1
False God 1
Florida!!! (feat. Florence + The Machin